Teste Radar

in questo file viene eseguito un test sul funzionamento del radar

In [2]:
import carla
import time

client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()

In [3]:

def spawn_vehicle(vehicle_index=0, spawn_index=0, x_offset=0, y_offset=0, pattern='vehicle.*model3*') -> carla.Vehicle:
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    spawn_point.location.x += x_offset
    spawn_point.location.y += y_offset
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

# Spawn target vehicle
target_vehicle = spawn_vehicle(x_offset=-50)

time.sleep(5)

# Spawn the vehicle
ego_vehicle = spawn_vehicle()

# Add the radar sensor
radar_bp = world.get_blueprint_library().find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '10')  # Horizontal field of view
radar_bp.set_attribute('vertical_fov', '10')    # Vertical field of view
radar_bp.set_attribute('range', '20')           # Maximum range

radar_transform = carla.Transform(carla.Location(x=2.0, z=1.0))
radar = world.spawn_actor(radar_bp, radar_transform, attach_to=ego_vehicle)

# Variable to store the minimum TTC
min_ttc = float('inf')

def radar_callback(data: carla.RadarMeasurement):
    global min_ttc, min_distance
    min_ttc = float('inf')

    for detection, i in zip(data, range(len(data))):
        absolute_speed = abs(detection.velocity)

        # Calculate TTC
        if absolute_speed != 0:
            ttc = detection.depth / absolute_speed
            print(ttc)
            if ttc < min_ttc:
                min_ttc = ttc

# Register the radar callback
radar.listen(radar_callback)
i=0
try:
    while True:
        # TTC threshold (e.g., 2 seconds)
        ttc_threshold = 1.0

        # Braking control
        if min_ttc < ttc_threshold:
            control = carla.VehicleControl()
            control.brake = 1.0  # Maximum braking
            ego_vehicle.apply_control(control)
            print("Emergency braking activated!")
            i += 1
            if i>10:
                break
        else:
            control = carla.VehicleControl()
            control.throttle = 0.5  # Maintain constant speed
            ego_vehicle.apply_control(control)
        
        time.sleep(0.05)
    time.sleep(2)
except KeyboardInterrupt:
    print("Keyboard interrupt detected.")
finally:
    radar.stop()
    radar.destroy()
    ego_vehicle.destroy()
    target_vehicle.destroy()


0.0009870932751597128
379.1690350401875
166.1038897218956
197.79113711605103
261.8025557877633
210.40017462123237
203.75396159396618
134.00949077332973
122.22525233687179
196.3898980202139
126.36795250837591
114.69839154865848
191.0486493937721
180.9479394560875
81.33113117103088
190.17248222877907
140.01603120416175
218.23379180970144
98.77584469693865
231.12615290654875
217.6546066389062
309.52941779813244
415.99010030677374
365.872832048154
389.0805988025614
227.33028670497927
942.438092349795
489.72901797802405
1114.0738529663586
924.601650358046
1256.2638895938207
1795.3133826855646
955.1850008174971
1861.102957312495
2410.902791153519
3854.0568414253203
1627.7954252019981
3891.743667502905
17583.24051219174
12129.150495436268
177408.22152741512
158888.4977684026
137709.28075652677
12572.941708557764
5010.872452944156
6310.23569031348
3167.39932933493
2344.4091238320493
2206.648998604651
2857.758574133805
2376.9499172007845
2181.5107901186793
1466.9428836119012
1429.0528881223534
